# 1. 순환 신경망(Recurrent Neural Network)
- [RNN](https://wikidocs.net/22886)
- 입력과 출력을 시퀀스 단위로 처리하는 시퀀스 모델
- 시퀀스 : 번역하고자 하는 단어의 문장
- 연속적인 데이터를 NN에 하나씩 순차적으로 넣어 처리하는 모델


### 1-1. RNN 동작 방식
- 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 다시 출력층 방향으로 보내면서 은닉층 노드의 다음 계산의 입력으로 보내는 것이 특징
- 셀(cell) : 은닉층에서 활성화 함수를 통해 나온 결과를 내보내는 역할을 하는 노드 이전의 값을 기억하려고 하는 일종의 메모리 역할 수행
- 은닉 상태(hidden state):  셀이 출력층 방향 또는 다음 시점인 t+1의 자신에게 보내는 값

```
rnn = torch.nn.RNN(input_size, hidden_size)
outputs, state = rnn(input_data)
# state = hidden size
```

### 1-2. input size
- 단어가 입력되면 각 글자를 벡터의 형태로 변환하여 one-hot encoding 해주는 과정이 필요
- 'hello'
  - h = [1,0,0,0]
  - e = [0,1,0,0]
  - l = [0,0,1,0]
  - o = [0,0,0,1]
- input_size = 4
- input_data의 세번쨰 차원으로 입력

### 1-3. Hidden State Size

- hidden state의 size는 output shape의 세번째 차원
- output size와 같음
- 셀에서 연산된 결과를 두 가지로 나눠 하나는 output으로 출력되고, 다른 하나는 hidden state로 다음 step에 그대로 전해짐

### 1-4. Sequence Length
- Sequence가 총 몇개인지를 나타냄
- 'hello'
    - x0 = [1,0,0,0]
    - x1 = [0,1,0,0]
    - x2 = [0,0,1,0]
    - x3 = [0,0,1,0]
    - x4 = [0,0,0,1]
- 'hello'를 입력으로 보내면 sequence length는 5
- Pytorch에서는 모델이 sequence length를 알아서 파악하기 때문에 파라미터로 전달해 줄 필요가 없음

### 1-5. batch size
- 여러 데이터를 묶어 하나의 batch로 만들어 학습을 진행
- [h,e,l,l,o][e,o,l,l,l][l,l,e,e,l] 처럼 h,e,l,o를 가지고 만들 수 있는 데이터 중 3개를 하나의 batch로 묶어 학습을 진행함
- batch size를 모델에서 자동으로 파악하고 output data, input data에서의 첫번째 차원에 위치함

In [1]:
import torch
import numpy as np
from torch.nn import RNN
import torch.nn as nn

import torch.optim as optim

In [2]:
input_size = 4
hidden_size = 2
epochs = 100

h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

input_data_np = np.array([[h,e,l,l,o],[e,o,l,l,l],[l,l,e,e,l]], dtype=np.float32)

input_data = torch.Tensor(input_data_np)

rnn = RNN(input_size, hidden_size)
outputs, state = rnn(input_data)

In [3]:
state

tensor([[[ 0.4202,  0.4924],
         [ 0.3660,  0.7304],
         [-0.4599,  0.7581],
         [-0.4599,  0.7581],
         [-0.0087,  0.7611]]], grad_fn=<StackBackward0>)

In [10]:
# helloworld
test = 'hello! world'
# string_set = ['h','e','l','o','w','r','d']
string_set = list(set(test))
print(string_set)

string_dic = {c: i for i, c in enumerate(string_set)}
print(string_dic)

input_size = len(string_dic)
print(input_size)
hidden_size = len(string_dic)
print(hidden_size)
learning_rate = 0.1

test_idx = [string_dic[c] for c in test]
print(test_idx)
x_data = [test_idx[:]]
print(x_data)
x_one_hot = [np.eye(input_size)[x] for x in x_data]
print(x_one_hot)
y_data = [test_idx[:]]
print(y_data)

X = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

['h', '!', 'w', ' ', 'l', 'r', 'e', 'o', 'd']
{'h': 0, '!': 1, 'w': 2, ' ': 3, 'l': 4, 'r': 5, 'e': 6, 'o': 7, 'd': 8}
9
9
[0, 6, 4, 4, 7, 1, 3, 2, 7, 5, 4, 8]
[[0, 6, 4, 4, 7, 1, 3, 2, 7, 5, 4, 8]]
[array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]])]
[[0, 6, 4, 4, 7, 1, 3, 2, 7, 5, 4, 8]]


In [11]:
# RNN
# CrossEntropyLoss()
# Adam
# epoch: 100
# loss를 출력

rnn = RNN(input_size, hidden_size)
loss_fun = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)


In [12]:
for i in range(100):
    optimizer.zero_grad()
    outputs, state = rnn(X)
    loss = loss_fun(outputs.view(-1, input_size), y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    print(result)
    result_str = ''.join([string_set[ch] for ch in np.squeeze(result)])
    print(i, 'loss:', loss)
    print('prediction:', result, 'prediction_str:', result_str)


[[3 3 3 3 3 3 7 3 3 3 3 3]]
0 loss: tensor(2.2635, grad_fn=<NllLossBackward0>)
prediction: [[3 3 3 3 3 3 7 3 3 3 3 3]] prediction_str:       o     
[[3 6 3 3 7 7 7 3 7 3 3 7]]
1 loss: tensor(2.0561, grad_fn=<NllLossBackward0>)
prediction: [[3 6 3 3 7 7 7 3 7 3 3 7]] prediction_str:  e  ooo o  o
[[3 6 4 4 7 1 7 3 7 3 4 8]]
2 loss: tensor(1.8555, grad_fn=<NllLossBackward0>)
prediction: [[3 6 4 4 7 1 7 3 7 3 4 8]] prediction_str:  ello!o o ld
[[4 6 4 4 7 1 7 3 7 4 4 8]]
3 loss: tensor(1.6793, grad_fn=<NllLossBackward0>)
prediction: [[4 6 4 4 7 1 7 3 7 4 4 8]] prediction_str: lello!o olld
[[4 6 4 4 7 1 3 4 7 4 4 8]]
4 loss: tensor(1.5399, grad_fn=<NllLossBackward0>)
prediction: [[4 6 4 4 7 1 3 4 7 4 4 8]] prediction_str: lello! lolld
[[0 6 4 4 7 1 4 2 7 4 4 8]]
5 loss: tensor(1.4313, grad_fn=<NllLossBackward0>)
prediction: [[0 6 4 4 7 1 4 2 7 4 4 8]] prediction_str: hello!lwolld
[[0 6 4 4 7 1 4 2 7 4 4 8]]
6 loss: tensor(1.3380, grad_fn=<NllLossBackward0>)
prediction: [[0 6 4 4 7 1 4 2 7 4

# 2. RNN의 단점
- 입력과 출력이 고정
- 기울기 소실
- 단점을 극복하기 위해 RNN의 발전 형태인 LSTM과 GRU를 사용(문제를 완벽히 해결하지 못함)